In [29]:
from tensorflow.keras.preprocessing.text import one_hot

In [30]:
sent=[
    'the glass of milk',
    'the glass of juice',
    'the cup of tea',
    'I am a good boy',
    'I am a good developer',
    'understand the meaning of words',
    'your videos are good'
]

In [31]:
voc_size=1000

In [32]:
one_hot_rep=[one_hot(word,voc_size) for word in sent]
one_hot_rep

[[518, 322, 676, 971],
 [518, 322, 676, 938],
 [518, 408, 676, 297],
 [942, 513, 872, 593, 145],
 [942, 513, 872, 593, 82],
 [50, 518, 28, 676, 838],
 [450, 561, 612, 593]]

In [33]:
#word Embeding
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential

In [34]:
import numpy as np

In [35]:
sent_length=8
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
embedded_docs

array([[  0,   0,   0,   0, 518, 322, 676, 971],
       [  0,   0,   0,   0, 518, 322, 676, 938],
       [  0,   0,   0,   0, 518, 408, 676, 297],
       [  0,   0,   0, 942, 513, 872, 593, 145],
       [  0,   0,   0, 942, 513, 872, 593,  82],
       [  0,   0,   0,  50, 518,  28, 676, 838],
       [  0,   0,   0,   0, 450, 561, 612, 593]], dtype=int32)

In [36]:
#fearure rep
dim=10

In [39]:
model=Sequential()
model.add(Embedding(voc_size,10,input_length=sent_length))
model.compile('adam','mse')

In [38]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.predict(embedded_docs[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step


array([[ 0.04951427, -0.01391337, -0.02071459,  0.02517286,  0.03354194,
         0.03711954,  0.03847137,  0.00386987,  0.00713773, -0.00378601],
       [ 0.04951427, -0.01391337, -0.02071459,  0.02517286,  0.03354194,
         0.03711954,  0.03847137,  0.00386987,  0.00713773, -0.00378601],
       [ 0.04951427, -0.01391337, -0.02071459,  0.02517286,  0.03354194,
         0.03711954,  0.03847137,  0.00386987,  0.00713773, -0.00378601],
       [ 0.04951427, -0.01391337, -0.02071459,  0.02517286,  0.03354194,
         0.03711954,  0.03847137,  0.00386987,  0.00713773, -0.00378601],
       [-0.04742033, -0.00878723, -0.00841645, -0.00553045,  0.04414325,
        -0.03372041, -0.0342716 ,  0.0050557 , -0.0415412 , -0.03887444],
       [-0.02623183, -0.00968397, -0.01685248,  0.03988291,  0.00403266,
        -0.04113602,  0.00755961,  0.01120243, -0.04364475, -0.00266569],
       [-0.03758251,  0.00366342, -0.02487391,  0.00819648,  0.0004968 ,
         0.00421122,  0.01151143,  0.02627843